# TDD with Ansible

# Topics

* What is TDD?
* What is Ansible?
* Why TDD with Ansible?
* What is Molecule?
* DEMO 🙀

# What is TDD?

![tdd-cycle](img/tdd-cycle.png)

The very core concepts are to set expectations on the code your're gonna write before writing it and improve it in a **fast feedback loop**

# What is Ansible?

A tool for doing this:
* Orchestration
* Configuration Management
* Application Deployment
* Provisioning
* Continuos Delivery
* Security Compliance

On one or more of these:
* Firewall
* Load Balancers
* Applications
* Containers
* Clouds
* Servers
* Infrastructure
* Storage
* Network Devices

The only requirement on the target devices is having Python installed.

![Ansible Platform](img/platform.png)

# Ansible Core is usable and open source

`pip install ansible`

# Ansible Primer
* The inventory is a file containing your device fleet
* Any device with Python installed can be managed
* Your devices are separated from the automation "scripts" called **Playbooks**
* A **Role** it's a way to write a sort of reusable **playbook**


# Extend Ansible
* Ansible capabilities can be extended using Python: by writing an Ansible's Module
* **Roles** and **Playbooks** can be packed and distributed as **Ansible Collections**

# Basic commands

* `ansible-playbook -i inventory playbook.yml`
* `ansible-galaxy collection install -vr requirements.yml`

An example of `requirements.yml`
```yaml
collections:
  - community.general
  - name: kubernetes.core
    version: "==2.4.0"
  - containers.podman
  ```

# Why TDD in Ansible?

*I'm not a great programmer; I'm just a good programmer with great habits* 

Ken Beck.

# Are Ansible playbooks code?

No, but they can be developed using the same *great habits*.

We still need **fast feedback loops**.

We really need **disposable sandboxes** for testing.

We also need to **protect against regression** in our playbooks.

Specifically:
* State messed up by previous playbook executions
* Task clobbering the state set by previous tasks
* Task silently failing or not working as intended
* Non-idempotent tasks.

Idempontency: do not redo stuff already done before so that a "script" can be invoked many time without harm.

# What is Molecule?

It's a simple but powerful testing framework for Ansible

It provides:
* CLI command for testing a "Scenario"
* Scaffolding commands to write a Scenario

Scenarios:
* Are configurations and entry points (playbooks) to test **Roles** and **Collections**

```tree
roles/common_stuff
├── defaults
├── files
├── handlers
├── meta
├── molecule
│   └── common_stuff_unit_test
│       ├── converge.yml
│       ├── molecule.yml
│       └── verify.yml
├── tasks
```

The previous directory structure is a **Role**.

**By convention** `ansible-playbook` expects this.

The entry point is `roles/common_stuff/tasks/main.yml`

# Molecule file

```yaml
dependency:
  name: galaxy
driver:
  name: podman
platforms:
  - name: instance
    image: quay.io/centos/centos:stream8
    pre_build_image: true
provisioner:
  name: ansible
verifier:
  name: ansible
```

# Converge file

```yaml
- name: Converge
  hosts: all
  tasks:
    - name: "Unit test converge"
      ansible.builtin.debug:
        msg: "This role can have unit tests since has not dependencies."
    - name: "Include tme.common_stuff"
      ansible.builtin.include_role:
        name: "tme.common_stuff"
```

# Verify file

```yaml
- name: Verify
  hosts: all
  gather_facts: false
  tasks:
  - name: "Is file present"
    stat:
      path: /tmp/common.txt
    register: st
  - name: "Example assertion: check the file existence"
    assert:
      that: st.stat.exists
```

# DEMO

https://github.com/keobox/test_molecule_monorepo/tree/upgrade

# Take home

With Molecule you can

* Write unit tests for a Role
* Write integrations tests for many Roles
* Check idempotency

### Basic TDD cycle

* molecule list
* molecule create -s my_scenario
* **write test code in verify.yml**
* molecule verify -s my_scenario <span style="color:red">**TEST FAIL**</span>
* **write code in roles**
* include roles in converge.yml
* molecule converge -s my_scenario
* molecule verify -s my_scenario <span style="color:green">**TEST PASS**</span>

### Idempotency checks

* molecule list
* molecule test -s my_scenario

By default **test** is equal to **destroy, create, converge, verify + idempotency check**.

In our toy project the file should not be created if is already created.

**NOTE: "molecule test" start a fresh container and I use it as Idempotency and Regression test after the TDD cycle**

A useful trick for integration testing of multiple roles in a repository:

In `molecule/my_scenario/molecule.yml`

```yaml
provisioner:
  name: ansible
  log: true
  env:
    ANSIBLE_ROLES_PATH: "../../roles"
```

# What if my code is in a Collection?

In this case is not necessary to to add `ANSIBLE_ROLES_PATH` in `molecule/my_scenario/molecule.yml` but the TDD cycle is slightly different.

### TDD cycle for collection.

* molecule list
* molecule create -s my_scenario
* **write test code in verify.yml**
* molecule verify -s my_scenario <span style="color:red">**TEST FAIL**</span>
* **write code in roles**
* <span style="color:blue">**ansible-galaxy collection install . --force**</span>
* <span style="color:blue">include **Roles** from **Collection** in converge.yml</span>
* molecule converge -s my_scenario
* molecule verify -s my_scenario <span style="color:green">**TEST PASS**</span>

# And that's it!

Cesare Placanica

Senior Software Engineer @ RedHat